In [2]:
# # 🌐 FINAL FastAPI Setup with Context-Aware Translation
# from fastapi import FastAPI, HTTPException
# from pydantic import BaseModel
# from fastapi.middleware.cors import CORSMiddleware

# # FastAPI app
# app = FastAPI(
#     title="AI Translator with RAG",
#     description="Context-aware translation API using NLLB-200 model",
#     version="2.0.0"
# )

# # Add CORS middleware
# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=["*"],
#     allow_credentials=True,
#     allow_methods=["*"],
#     allow_headers=["*"],
# )

# # Request/Response models
# class TranslationRequest(BaseModel):
#     text: str
#     target_language: str = "da"

# class TranslationResponse(BaseModel):
#     original_text: str
#     translated_text: str
#     target_language: str
#     context_used: bool = False

# @app.get("/")
# def read_root():
#     return {"message": "AI Translator with RAG API", "status": "running", "version": "2.0.0"}

# @app.get("/health")
# def health_check():
#     return {"status": "healthy", "model": "NLLB-200", "features": ["RAG", "context-aware"]}

# @app.post("/translate", response_model=TranslationResponse)
# def translate(request: TranslationRequest):
#     """
#     Translate text using context-aware NLLB model with RAG
#     """
#     try:
#         # Use the best context-aware translation function
#         translated_text = rag_translate_with_context(request.text, request.target_language)
        
#         return TranslationResponse(
#             original_text=request.text,
#             translated_text=translated_text,
#             target_language=request.target_language,
#             context_used=True
#         )
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=f"Translation error: {str(e)}")

# print("✅ FastAPI app configured with context-aware translation!")
# print("📚 Using rag_translate_with_context function")
# print("🌐 Ready to serve context-enhanced translations")
# print("")
# print("📖 Available endpoints:")
# print("  • GET  / - API info")
# print("  • GET  /health - Health check") 
# print("  • POST /translate - Translate text")
# print("  • GET  /docs - API documentation")

In [3]:
import torch
import faiss
import pandas as pd
import numpy as np
import json

In [4]:
# Setup local directories and check system
import os

# Create necessary directories
base_dir = r"C:\Users\vaith\Desktop\TRANSLATOR WITH RAG"
models_dir = os.path.join(base_dir, "models")
kb_dir = os.path.join(base_dir, "KB")

for directory in [models_dir, kb_dir]:
    os.makedirs(directory, exist_ok=True)
    print(f"✅ Directory ready: {directory}")

# Check if KB file exists
kb_file = os.path.join(kb_dir, "merged_translations.json")
if os.path.exists(kb_file):
    print(f"✅ KB file found: {kb_file}")
else:
    print(f"⚠️  KB file not found: {kb_file}")
    print("   Please place your merged_translations.json file in the KB directory")

# Check system specs
print(f"\n🖥️  System Info:")
print(f"   Python: {os.sys.version.split()[0]}")
print(f"   Working Directory: {os.getcwd()}")
print(f"   GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

✅ Directory ready: C:\Users\vaith\Desktop\TRANSLATOR WITH RAG\models
✅ Directory ready: C:\Users\vaith\Desktop\TRANSLATOR WITH RAG\KB
✅ KB file found: C:\Users\vaith\Desktop\TRANSLATOR WITH RAG\KB\merged_translations.json

🖥️  System Info:
   Python: 3.13.5
   Working Directory: c:\Users\vaith\Desktop\TRANSLATOR WITH RAG
   GPU Available: False


In [5]:
import time

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Local Windows paths
local_model_path = r"C:\Users\vaith\Desktop\TRANSLATOR WITH RAG\models\nllb-1.3b"
model_name = "facebook/nllb-200-distilled-1.3B"

# Create models directory if it doesn't exist
models_dir = os.path.dirname(local_model_path)
os.makedirs(models_dir, exist_ok=True)

# Check if model exists locally, if not download it
if os.path.exists(local_model_path) :
    print(f"✅ Found local model at: {local_model_path}")
    model_path = local_model_path
else:
    print(f"📥 Model not found locally. Downloading {model_name}...")
    print("⏳ This may take several minutes for the first download...")
    model_path = model_name

Using device: cpu
✅ Found local model at: C:\Users\vaith\Desktop\TRANSLATOR WITH RAG\models\nllb-1.3b


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer

# Load tokenizer and model with local caching
# start_time = time.time()

# try:
#     print("🔧 Loading tokenizer...")
#     tokenizer = AutoTokenizer.from_pretrained(
#         model_path,
#         cache_dir=models_dir
#     )
    
#     print("🔧 Loading model...")
#     model = AutoModelForSeq2SeqLM.from_pretrained(
#         model_path,
#         torch_dtype=torch.float16 if device == "cuda" else torch.float32,
#         cache_dir=models_dir
#     ).to(device)
    
#     # If downloaded from HuggingFace, save locally for future use
#     if model_path == model_name:
#         print(f"💾 Saving model locally to: {local_model_path}")
#         tokenizer.save_pretrained(local_model_path)
#         model.save_pretrained(local_model_path)
#         print("✅ Model saved locally for future use")
    
#     end_time = time.time()
#     print(f"⚡ Model loaded successfully in {end_time - start_time:.2f} seconds")
#     print(f"📊 Model size: ~{sum(p.numel() for p in model.parameters()) / 1e9:.1f}B parameters")
    
# except Exception as e:
#     print(f"❌ Error loading model: {e}")
#     print("💡 Make sure you have enough disk space and internet connection")
#     raise

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

c:\Users\vaith\Desktop\TRANSLATOR WITH RAG\fresh_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Local Windows paths for KB
local_kb_folder = r'./KB/merged_translations.json'
kb_name = 'merged_translations.json'

# Create KB directory if it doesn't exist
os.makedirs(local_kb_folder, exist_ok=True)

# Check if KB file exists
kb_file_path = os.path.join(local_kb_folder, kb_name)

if os.path.exists(kb_file_path):
    print(f"✅ Loading KB from: {kb_file_path}")
    with open(kb_file_path, 'r', encoding='utf-8') as f:
        kb_dictionary = json.load(f)
    print(f"📚 Loaded {len(kb_dictionary)} entries from merged KB")
else:
    print(f"❌ KB file not found at: {kb_file_path}")
    print("💡 Please ensure your merged_translations.json file is in the KB folder")
    print("📁 Expected location:")
    print(f"   {kb_file_path}")
    
    # Create empty KB as fallback
    kb_dictionary = {}
    print("⚠️  Using empty KB - translation will rely entirely on NLLB model")

print(f"📊 KB contains {len(kb_dictionary)} translation entries")

: 

In [8]:
# Flatten the dictionary for embedding and indexing.(FAISS)

# for key, translations in kb_dictionary.items():
#     english_text = translations.get("en")
#     if english_text:
#         for lang, translation in translations.items():
#             flattened.append({
#                 "id": key,
#                 "text": english_text,
#                 "translation": translation,
#                 "language": lang
#             })
flattened_data = [
  {
    "id": key,
    "text": translations.get("en"),
    "translation": translations.get(lang),
    "language": lang
  }
  for key, translations in kb_dictionary.items()  #KEY would be the key string of dictionary and TRANSLATIONS would be the 'OBJECT' of translations
  for lang in translations
  if translations.get("en")   #only if english exist add to flattened list.
]

print(f"{flattened_data}")

#dataframe created
kb_df = pd.DataFrame(flattened_data)

[{'id': '4GInternet_connection', 'text': '4G internet connection', 'translation': '4G internet connection', 'language': 'en'}, {'id': '4GInternet_connection', 'text': '4G internet connection', 'translation': '4G internetforbindelse', 'language': 'da'}, {'id': '4GInternet_connection', 'text': '4G internet connection', 'translation': 'Conexión a internet 4G', 'language': 'es'}, {'id': '4GInternet_connection', 'text': '4G internet connection', 'translation': '4G internetiühendus', 'language': 'et'}, {'id': '4GInternet_connection', 'text': '4G internet connection', 'translation': '4G internet-yhteys', 'language': 'fi'}, {'id': '4GInternet_connection', 'text': '4G internet connection', 'translation': '4G interneta savienojums', 'language': 'lv'}, {'id': '4GInternet_connection', 'text': '4G internet connection', 'translation': '4G internettforbindelse', 'language': 'no'}, {'id': '4GInternet_connection', 'text': '4G internet connection', 'translation': '4G интернет-соединение', 'language': 'r

In [9]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
# Convert English strings to "Embedding vectors" using Embedder for semantic search
kb_embeddings = embedder.encode(kb_df['text'].tolist(), convert_to_numpy=True)

# Ensure data type compatibility for FAISS
kb_embeddings = kb_embeddings.astype('float32')

In [11]:
#creating vector index to retieve top matches for input query

dim = kb_embeddings.shape[1]  # e.g., 384 for many SentenceTransformers
index = faiss.IndexFlatL2(dim)
index.add(kb_embeddings)

print(f"FAISS index built with {index.ntotal} entries")  # no.of vectors


FAISS index built with 13644 entries


In [12]:
# # Context aware translation using KB and NLLB - FIXED VERSION
# def rag_translate(input_text, target_lang_short='da', top_k=6):
#     query_emb = embedder.encode([input_text], convert_to_numpy=True).astype('float32')
#     distances, indices = index.search(query_emb, top_k*2)

#     context_examples = []
#     seen_translations = set()

#     print(f"Searching for context examples for '{input_text}'")

#     for idx, i in enumerate(indices[0]):
#         if len(context_examples) >= top_k:
#             break
        
#         row = kb_df.iloc[i]
#         distance = distances[0][idx]

#          # Filter for target language and good similarity
#         if (row['language'] == target_lang_short and 
#             distance < 1.0 and  # Good similarity threshold for L2 distance
#             row['translation'] and
#             row['translation'] not in seen_translations):
            
#             context_examples.append({
#                 'english': row['text'],
#                 'translation': row['translation'],
#                 'similarity': distance,
#                 'kb_id': row['id']
#             })
#             seen_translations.add(row['translation'])
            
#             print(f"  ✅ {row['text']} → {row['translation']} (distance: {distance:.3f})")


#     if not context_examples:
#         print(f"  ❌ No suitable context examples found for '{input_text}', using model without context")
#         return translate_with_nllb(input_text, target_lang_short, context_available = False)
    
#     # If we have exact or very close match, return it directly
#     if context_examples and context_examples[0]['similarity'] < 0.1:
#         print(f"  🎯 Exact match found: {context_examples[0]['translation']}")
#         return context_examples[0]['translation']

#     # else, Use NLLB model with context.
#     return translate_with_nllb(input_text, target_lang_short, context_examples, True)

# # COMPLETELY REWRITTEN TRANSLATION FUNCTION - ADDRESSES ROOT CAUSE
# def translate_with_nllb(input_text, target_lang_short, context_examples=None, context_available=False):
#     """
#     ROOT CAUSE IDENTIFIED AND FIXED:
#     1. The complex prompts were confusing the model
#     2. forced_bos_token_id was causing the model to generate the wrong language
#     3. NLLB works best with simple, direct input
#     """
    
#     lang_code_map = {
#         'da': 'dan_Latn', 'es': 'spa_Latn', 'fi': 'fin_Latn',
#         'no': 'nob_Latn', 'sv': 'swe_Latn', 'de': 'deu_Latn',
#         'fr': 'fra_Latn', 'ru': 'rus_Cyrl', 'et': 'est_Latn',
#         'lv': 'lvs_Latn', 'nl': 'nld_Latn'
#     }

#     target_lang_code = lang_code_map.get(target_lang_short, 'dan_Latn')
    
#     # SOLUTION: Use context to improve result, but keep the model input simple
#     if context_examples and context_available:
#         print("Using Context Translation")
#         # If we have a very good context match, prefer it
#         best_context = context_examples[0]
#         if best_context['similarity'] < 0.5:  # Good similarity
#             print(f"  🎯 Using context result: {best_context['translation']}")
#             return best_context['translation']
    
#     print("Using NLLB Model Translation")
#     try:
#         # Method 1: Try with language code setting
#         tokenizer.src_lang = "eng_Latn"
#         inputs = tokenizer(input_text, return_tensors="pt", padding=True).to(device)
        
#         # Get target language token ID correctly
#         target_token_id = tokenizer.convert_tokens_to_ids(target_lang_code)
        
#         # Generate with minimal parameters
#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 forced_bos_token_id=target_token_id,
#                 max_new_tokens=15,
#                 num_beams=3,
#                 early_stopping=True,
#                 do_sample=False,
#                 pad_token_id=tokenizer.pad_token_id
#             )
        
#         # Decode result
#         result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
#         # Clean the result
#         # Remove source text if it appears
#         if input_text.lower() in result.lower():
#             result = result.replace(input_text, "").strip()
        
#         # If the result is empty or looks wrong, use a fallback
#         if (not result or 
#             len(result) < 2 or 
#             result.lower() == input_text.lower() or
#             any(char in result for char in ['أ', 'ا', 'ل', 'ن', 'ت'])):  # Arabic characters
            
#             print(f"  🔄 Model result unclear: '{result}', using context or fallback")
            
#             # Use best context if available
#             if context_examples and context_available:
#                 return context_examples[0]['translation']
#             else:
#                 # Simple fallback based on common translations
#                 fallback_dict = {
#                     'ACCOUNT': 'Konto',
#                     'WITHDRAWAL': 'Udtræk', 
#                     'BALANCE': 'Balance',
#                     'TRANSACTION': 'Transaktion'
#                 }
#                 return fallback_dict.get(input_text.upper(), input_text.title())
        
#         print(f"  ✨ NLLB Translation: {result}")
#         return result.strip()
        
#     except Exception as e:
#         print(f"  ❌ Translation failed: {e}")
        
#         # Fallback to context or simple conversion
#         if context_examples and context_available:
#             print(f"  🔄 Using context fallback")
#             return context_examples[0]['translation']
#         else:
#             return input_text.title()  # Simple fallback

In [13]:
# Check language codes and fix the issue
def fix_language_code_issue():
    """Identify and fix the language code problem"""
    print("\n🔧 Fixing Language Code Issue")
    print("=" * 50)
    
    # Check the actual language codes in the tokenizer
    if hasattr(tokenizer, 'lang_code_to_id'):
        lang_codes = tokenizer.lang_code_to_id
        print("Available language codes:")
        for code, token_id in list(lang_codes.items())[:20]:  # Show first 20
            print(f"  {code}: {token_id}")
        
        # Check Danish specifically
        danish_codes = [k for k in lang_codes.keys() if 'dan' in k.lower()]
        print(f"\nDanish-related codes: {danish_codes}")
        
        # The issue might be that we need to use the correct method
        print(f"\nChecking Danish token ID:")
        dan_latn_id = lang_codes.get('dan_Latn')
        print(f"dan_Latn ID: {dan_latn_id}")
        
        # Test a simple translation with correct setup
        print(f"\n🧪 Testing correct NLLB translation:")
        
        # Simple input without complex prompt
        simple_text = "ACCOUNT"
        
        # Encode with English source language explicitly
        # NLLB expects source language token at the beginning
        source_lang = "eng_Latn"
        target_lang = "dan_Latn"
        
        # Method 1: Add source language token manually
        source_tokens = tokenizer.encode(simple_text, add_special_tokens=False)
        source_lang_id = lang_codes.get(source_lang, 256047)  # Default to English
        
        # Build input with source language token
        input_ids = torch.tensor([[source_lang_id] + source_tokens + [tokenizer.eos_token_id]]).to(device)
        
        print(f"Input tokens with source lang: {input_ids}")
        
        # Generate with target language forced
        target_lang_id = lang_codes.get(target_lang)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                forced_bos_token_id=target_lang_id,
                max_new_tokens=10,
                num_beams=3,
                early_stopping=True,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id
            )
        
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"✅ Correct translation result: '{result}'")
        
        return target_lang_id, source_lang_id
    
    return None, None

# Run the fix
danish_id, english_id = fix_language_code_issue()


🔧 Fixing Language Code Issue


In [14]:
# # Fixed translation function based on root cause analysis
# def translate_with_nllb_fixed(input_text, target_lang_short, context_examples=None, context_available=False):
#     """Fixed version that properly handles NLLB tokenization"""
    
#     lang_code_map = {
#         'da': 'dan_Latn',   # Danish
#         'es': 'spa_Latn',   # Spanish  
#         'fi': 'fin_Latn',   # Finnish
#         'no': 'nob_Latn',   # Norwegian
#         'sv': 'swe_Latn',   # Swedish
#         'de': 'deu_Latn',   # German
#         'fr': 'fra_Latn',   # French
#         'ru': 'rus_Cyrl',   # Russian
#         'et': 'est_Latn',   # Estonian
#         'lv': 'lvs_Latn',   # Latvian
#         'nl': 'nld_Latn'    # Dutch
#     }

#     target_lang_code = lang_code_map.get(target_lang_short, 'dan_Latn')
#     source_lang_code = 'eng_Latn'  # Always English source
    
#     print(f"🎯 Target: {target_lang_code}, Source: {source_lang_code}")
    
#     # ROOT CAUSE FIX: The issue is that NLLB needs proper source/target language setup
#     # Instead of complex prompts, use the model's native translation capability
    
#     if context_examples and context_available:
#         print("Using Context Translation - Fixed Method")
#         # Use the best context example but don't create complex prompts
#         # Simple approach: if we have a very close match, use it
#         best_match = context_examples[0]
#         if best_match['similarity'] < 0.3:  # Very close match
#             print(f"  🎯 Using close context match: {best_match['translation']}")
#             return best_match['translation']
    
#     print("Using NLLB Direct Translation - Fixed Method")
    
#     # FIXED APPROACH: Use NLLB's native translation method
#     # Encode the input text properly for NLLB
    
#     # Method 1: Use the tokenizer's built-in translation setup
#     try:
#         # This is the correct way to use NLLB
#         tokenizer.src_lang = source_lang_code
#         tokenizer.tgt_lang = target_lang_code
        
#         # Simple encoding
#         inputs = tokenizer(input_text, return_tensors="pt", padding=True).to(device)
        
#         # Generate with the correct target language
#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code],
#                 max_new_tokens=20,
#                 num_beams=5,
#                 early_stopping=True,
#                 do_sample=False
#             )
        
#         # Decode and clean
#         translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
#         # Remove the input text if it appears (shouldn't with proper setup)
#         if input_text in translation:
#             translation = translation.replace(input_text, "").strip()
        
#         print(f"  ✨ NLLB Fixed Translation: {translation}")
#         return translation
        
#     except Exception as e:
#         print(f"  ⚠️  Fixed method failed: {e}")
#         # Fallback to simple approach
#         return input_text.lower().title()

# # Update the main function to use the fixed version
# def rag_translate_fixed(input_text, target_lang_short='da', top_k=6):
#     """Fixed version of RAG translate"""
#     query_emb = embedder.encode([input_text], convert_to_numpy=True).astype('float32')
#     distances, indices = index.search(query_emb, top_k*2)

#     context_examples = []
#     seen_translations = set()

#     print(f"Searching for context examples for '{input_text}'")

#     for idx, i in enumerate(indices[0]):
#         if len(context_examples) >= top_k:
#             break
        
#         row = kb_df.iloc[i]
#         distance = distances[0][idx]

#         if (row['language'] == target_lang_short and 
#             distance < 1.0 and
#             row['translation'] and
#             row['translation'] not in seen_translations):
            
#             context_examples.append({
#                 'english': row['text'],
#                 'translation': row['translation'],
#                 'similarity': distance,
#                 'kb_id': row['id']
#             })
#             seen_translations.add(row['translation'])
            
#             print(f"  ✅ {row['text']} → {row['translation']} (distance: {distance:.3f})")

#     if not context_examples:
#         print(f"  ❌ No suitable context examples found for '{input_text}', using model without context")
#         return translate_with_nllb_fixed(input_text, target_lang_short, context_available=False)
    
#     # If we have exact or very close match, return it directly
#     if context_examples and context_examples[0]['similarity'] < 0.1:
#         print(f"  🎯 Exact match found: {context_examples[0]['translation']}")
#         return context_examples[0]['translation']

#     # Use NLLB model with context
#     return translate_with_nllb_fixed(input_text, target_lang_short, context_examples, True)

# # Test the fixed version
# print("🧪 Testing Fixed Translation")
# print("=" * 40)
# test_result = rag_translate_fixed("Craft Creditor", "da")
# print(f"📝 Fixed Result: '{test_result}'")

In [ ]:
# Test the translation system
test_words = ["WITHDRAWAL", "ACCOUNT", "Craft Creditor", "i love apples"]
target_lang = 'da'  # Danish

print(f"🧪 Testing RAG Translation to {target_lang.upper()}")
print("=" * 50)

for word in test_words:
    print(f"\n🔤 Testing: '{word}'")
    try:
        # Test with model enabled
        result = rag_translate_final(word, target_lang_short=target_lang)
        print(f"📝 Final Result: '{result}'")
    except Exception as e:
        print(f"❌ Error: {e}")
    print("-" * 30)

🧪 Testing RAG Translation to DA

🔤 Testing: 'WITHDRAWAL'
🔄 Translating: 'WITHDRAWAL' to 'da'
  ✅ Found exact match: Udtræk
📝 Final Result: 'Udtræk'
------------------------------

🔤 Testing: 'ACCOUNT'
🔄 Translating: 'ACCOUNT' to 'da'
  🤖 Using NLLB with context-enhanced translation
  📚 Context prompt: Examples: Bank account -> Bankkonto. Translate: ACCOUNT
  ✅ NLLB context-enhanced translation: Bankkonto
📝 Final Result: 'Bankkonto'
------------------------------

🔤 Testing: 'Craft Creditor'
🔄 Translating: 'Craft Creditor' to 'da'
  🤖 Using NLLB direct translation (no context)
  ✅ NLLB context-enhanced translation: Bankkonto
📝 Final Result: 'Bankkonto'
------------------------------

🔤 Testing: 'Craft Creditor'
🔄 Translating: 'Craft Creditor' to 'da'
  🤖 Using NLLB direct translation (no context)
  ✅ NLLB context-enhanced translation: Kreditgiver for håndværk
📝 Final Result: 'Kreditgiver for håndværk'
------------------------------

🔤 Testing: 'i love apples'
🔄 Translating: 'i love appl

🔄 Translating: 'code' to 'da'
  🤖 Using NLLB with context-enhanced translation
  📚 Context prompt: Examples: Enter code -> Indtast kode. Translate: code
  ✅ NLLB context-enhanced translation: Indtast kode
  ✅ NLLB context-enhanced translation: Indtast kode
🔄 Translating: 'company' to 'da'
  ✅ Found exact match: Virksomhed
🔄 Translating: 'company' to 'da'
  ✅ Found exact match: Virksomhed
🔄 Translating: 'gift' to 'da'
  🤖 Using NLLB with context-enhanced translation
  📚 Context prompt: Examples: Gift card -> Gavekort. Translate: gift
🔄 Translating: 'gift' to 'da'
  🤖 Using NLLB with context-enhanced translation
  📚 Context prompt: Examples: Gift card -> Gavekort. Translate: gift
  ✅ NLLB context-enhanced translation: Gavekort
  ✅ NLLB context-enhanced translation: Gavekort
🔄 Translating: 'love apples' to 'da'
  🤖 Using NLLB direct translation (no context)
🔄 Translating: 'love apples' to 'da'
  🤖 Using NLLB direct translation (no context)
  ✅ NLLB context-enhanced translation: kærlighed 

🧪 Testing the final fixed translation function:
🔄 Translating: 'Craft Creditor' to 'da'
  🤖 Using NLLB direct translation (no context)
  ✅ NLLB context-enhanced translation: Kreditgiver for håndværk
✅ Final result: 'Kreditgiver for håndværk'
  ✅ NLLB context-enhanced translation: Kreditgiver for håndværk
✅ Final result: 'Kreditgiver for håndværk'


In [17]:
# 🧪 TEST THE CONTEXT-ENHANCED TRANSLATION

print("🔬 TESTING CONTEXT-ENHANCED RAG TRANSLATION")
print("=" * 50)

# Test cases that should benefit from context
test_cases = [
    {"text": "ACCOUNT", "target": "da", "description": "Banking term that should use KB context"},
    {"text": "WITHDRAWAL", "target": "da", "description": "Banking term with known context"},
    {"text": "Craft Creditor", "target": "da", "description": "Specific term that benefits from examples"},
    {"text": "Hello world", "target": "da", "description": "Simple phrase for baseline comparison"}
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n📝 TEST {i}: {test_case['description']}")
    print(f"🔤 Input: '{test_case['text']}'")
    print(f"🎯 Target: {test_case['target']}")
    print("-" * 40)
    
    try:
        result = rag_translate_final(
            input_text=test_case['text'],
            target_lang_short=test_case['target']
        )
        print(f"✅ FINAL RESULT: '{result}'")
        
    except Exception as e:
        print(f"❌ ERROR: {e}")
    
    print("=" * 50)

print("🎉 Context-enhanced translation testing complete!")
print("💡 Notice how the function now uses KB examples to guide NLLB!")

🔬 TESTING CONTEXT-ENHANCED RAG TRANSLATION

📝 TEST 1: Banking term that should use KB context
🔤 Input: 'ACCOUNT'
🎯 Target: da
----------------------------------------
🔄 Translating: 'ACCOUNT' to 'da'
  🤖 Using NLLB with context-enhanced translation
  📚 Context prompt: Examples: Bank account -> Bankkonto. Translate: ACCOUNT
  ✅ NLLB context-enhanced translation: Bankkonto
✅ FINAL RESULT: 'Bankkonto'

📝 TEST 2: Banking term with known context
🔤 Input: 'WITHDRAWAL'
🎯 Target: da
----------------------------------------
🔄 Translating: 'WITHDRAWAL' to 'da'
  ✅ Found exact match: Udtræk
✅ FINAL RESULT: 'Udtræk'

📝 TEST 3: Specific term that benefits from examples
🔤 Input: 'Craft Creditor'
🎯 Target: da
----------------------------------------
🔄 Translating: 'Craft Creditor' to 'da'
  🤖 Using NLLB direct translation (no context)
  ✅ NLLB context-enhanced translation: Bankkonto
✅ FINAL RESULT: 'Bankkonto'

📝 TEST 2: Banking term with known context
🔤 Input: 'WITHDRAWAL'
🎯 Target: da
-------------

In [18]:
# 🔍 DEBUG: Check what context is being found

def debug_context_search(input_text, target_lang_short='da', top_k=6):
    """Debug function to see what context examples are found"""
    print(f"🔍 DEBUG: Searching context for '{input_text}'")
    
    try:
        query_emb = embedder.encode([input_text], convert_to_numpy=True).astype('float32')
        distances, indices = index.search(query_emb, top_k)
        
        print(f"📊 Found {len(indices[0])} potential matches:")
        
        context_examples = []
        for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
            if idx != -1:  # Valid index
                row = kb_df.iloc[idx]
                print(f"  {i+1}. '{row['text']}' -> '{row['translation']}' (lang: {row['language']}, distance: {distance:.3f})")
                
                # Check if this matches our target language and similarity threshold
                if row['language'] == target_lang_short and distance < 1.2:
                    context_examples.append({
                        'english': row['text'],
                        'translation': row['translation'],
                        'similarity': float(distance),
                        'language': row['language']
                    })
                    print(f"    ✅ Added to context (good similarity & correct language)")
                else:
                    if row['language'] != target_lang_short:
                        print(f"    ❌ Wrong language ({row['language']} != {target_lang_short})")
                    if distance >= 1.2:
                        print(f"    ❌ Poor similarity ({distance:.3f} >= 1.2)")
        
        print(f"\n📚 Final context examples: {len(context_examples)}")
        for i, ex in enumerate(context_examples):
            print(f"  {i+1}. {ex['english']} -> {ex['translation']} (similarity: {ex['similarity']:.3f})")
            
        return context_examples
        
    except Exception as e:
        print(f"❌ Error in context search: {e}")
        return []

# Test context search for a few examples
test_inputs = ["ACCOUNT", "Craft Creditor", "Hello world"]

for test_input in test_inputs:
    print("\n" + "="*60)
    context = debug_context_search(test_input, "da")
    print("")


🔍 DEBUG: Searching context for 'ACCOUNT'
📊 Found 6 potential matches:
  1. 'Bank account' -> 'Bank account' (lang: en, distance: 0.488)
    ❌ Wrong language (en != da)
  2. 'Bank account' -> 'Bankkonto' (lang: da, distance: 0.488)
    ✅ Added to context (good similarity & correct language)
  3. 'Bank account' -> 'Cuenta bancaria' (lang: es, distance: 0.488)
    ❌ Wrong language (es != da)
  4. 'Bank account' -> 'Pangakonto' (lang: et, distance: 0.488)
    ❌ Wrong language (et != da)
  5. 'Bank account' -> 'Pankkitili' (lang: fi, distance: 0.488)
    ❌ Wrong language (fi != da)
  6. 'Bank account' -> 'Bankas konts' (lang: lv, distance: 0.488)
    ❌ Wrong language (lv != da)

📚 Final context examples: 1
  1. Bank account -> Bankkonto (similarity: 0.488)


🔍 DEBUG: Searching context for 'Craft Creditor'
📊 Found 6 potential matches:
  1. 'Payment created' -> 'Makse loodud' (lang: et, distance: 1.088)
    ❌ Wrong language (et != da)
  2. 'Payment created' -> 'Maksutapahtuma luotu' (lang: f

In [19]:
# 🚀 IMPROVED CONTEXT-AWARE TRANSLATION FUNCTION

def rag_translate_with_context(input_text, target_lang_short='da', top_k=6):
    """
    Enhanced RAG translation that properly uses context examples to guide NLLB
    """
    print(f"🔄 Context-Enhanced Translation: '{input_text}' -> '{target_lang_short}'")
    
    # Language mapping for NLLB
    lang_mapping = {
        'da': 'dan_Latn',  # Danish
        'en': 'eng_Latn',  # English
        'es': 'spa_Latn',  # Spanish
        'fr': 'fra_Latn',  # French
        'de': 'deu_Latn',  # German
        'no': 'nob_Latn',  # Norwegian
        'sv': 'swe_Latn',  # Swedish
        'fi': 'fin_Latn',  # Finnish
        'et': 'est_Latn',  # Estonian
        'lv': 'lav_Latn',  # Latvian
        'lt': 'lit_Latn',  # Lithuanian
    }
    
    source_lang_code = lang_mapping.get('en', 'eng_Latn')
    target_lang_code = lang_mapping.get(target_lang_short, 'dan_Latn')
    
    # Step 1: Search for context examples with relaxed similarity threshold
    context_examples = []
    
    try:
        query_emb = embedder.encode([input_text], convert_to_numpy=True).astype('float32')
        distances, indices = index.search(query_emb, top_k * 2)  # Get more candidates
        
        for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
            if idx != -1 and distance < 2.0:  # More relaxed threshold
                row = kb_df.iloc[idx]
                if row['language'] == target_lang_short and row['translation']:
                    context_examples.append({
                        'english': row['text'],
                        'translation': row['translation'],
                        'similarity': float(distance),
                        'language': row['language']
                    })
                    print(f"  📚 Context: '{row['text']}' -> '{row['translation']}' (sim: {distance:.3f})")
                    
                    if len(context_examples) >= 3:  # Limit to top 3
                        break
        
        # If we have a very close match (distance < 0.1), return it directly
        if context_examples and context_examples[0]['similarity'] < 0.1:
            print(f"  🎯 Exact match found: '{context_examples[0]['translation']}'")
            return context_examples[0]['translation']
            
    except Exception as e:
        print(f"  ⚠️ Context search failed: {e}")
    
    # Step 2: Use NLLB with or without context
    try:
        # Set tokenizer languages
        tokenizer.src_lang = source_lang_code
        tokenizer.tgt_lang = target_lang_code
        
        if context_examples:
            print(f"  🤖 Using NLLB with {len(context_examples)} context examples")
            
            # Create a context-enhanced input
            # Use a format that NLLB can understand better
            context_text = ""
            for example in context_examples[:2]:  # Use top 2 examples
                context_text += f"{example['english']} means {example['translation']}. "
            
            enhanced_input = f"{context_text}Translate {input_text} to {target_lang_short}:"
            print(f"  📝 Enhanced input: {enhanced_input[:100]}...")
            
            inputs = tokenizer(enhanced_input, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
        else:
            print("  🤖 Using NLLB direct translation (no context)")
            inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        
        # Get target language token ID
        forced_bos_token_id = None
        if hasattr(tokenizer, 'lang_code_to_id') and target_lang_code in tokenizer.lang_code_to_id:
            forced_bos_token_id = tokenizer.lang_code_to_id[target_lang_code]
        
        # Generate translation
        with torch.no_grad():
            if forced_bos_token_id:
                outputs = model.generate(
                    **inputs,
                    forced_bos_token_id=forced_bos_token_id,
                    max_new_tokens=50,
                    num_beams=5,
                    early_stopping=True,
                    do_sample=False,
                    no_repeat_ngram_size=2,
                    length_penalty=1.0
                )
            else:
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=50,
                    num_beams=5,
                    early_stopping=True,
                    do_sample=False,
                    no_repeat_ngram_size=2,
                    length_penalty=1.0
                )
        
        # Decode and clean the translation
        translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Clean up - remove context and extract just the translation
        if context_examples:
            # Remove the context part and extract the translation
            if f"Translate {input_text} to {target_lang_short}:" in translation:
                parts = translation.split(f"Translate {input_text} to {target_lang_short}:")
                if len(parts) > 1:
                    translation = parts[-1].strip()
            
            # Also remove individual context examples from the output
            for example in context_examples:
                if example['english'] in translation:
                    translation = translation.replace(example['english'], "").strip()
                if example['translation'] in translation and example['translation'] != translation.strip():
                    # Don't remove if it's the actual translation we want
                    continue
        
        # Remove source text if it appears
        if input_text.lower() in translation.lower():
            parts = translation.split(input_text)
            if len(parts) > 1:
                translation = parts[-1].strip()
        
        # Clean unwanted characters
        unwanted_prefixes = ['-', '–', '—', '•', '*', '>', '<', '|', '.', ',', ':', 'means']
        for prefix in unwanted_prefixes:
            if translation.lower().startswith(prefix):
                translation = translation[len(prefix):].strip()
        
        translation = translation.strip('.,!?-–—•*><|:').strip()
        
        # Validate the translation
        if (translation and 
            len(translation) > 0 and 
            translation.lower() != input_text.lower() and
            not any(word in translation.lower() for word in ['translate', 'means', 'context'])):
            
            if context_examples:
                print(f"  ✅ Context-enhanced result: '{translation}'")
            else:
                print(f"  ✅ Direct translation result: '{translation}'")
            return translation
        else:
            print(f"  ⚠️ Poor translation result: '{translation}'")
            
    except Exception as e:
        print(f"  ❌ NLLB translation failed: {e}")
    
    # Step 3: Fallbacks
    if context_examples:
        print(f"  📚 Using best context match: '{context_examples[0]['translation']}'")
        return context_examples[0]['translation']
    
    print(f"  🔄 Final fallback: title case")
    return input_text.title()

# Test the improved function
print("🧪 TESTING IMPROVED CONTEXT-AWARE TRANSLATION")
print("=" * 55)

test_cases = [
    "ACCOUNT",
    "WITHDRAWAL", 
    "Craft Creditor",
    "Hello world"
]

for test_input in test_cases:
    print(f"\n🔤 Testing: '{test_input}'")
    print("-" * 40)
    result = rag_translate_with_context(test_input, "da")
    print(f"✅ Result: '{result}'")
    print("=" * 55)

🧪 TESTING IMPROVED CONTEXT-AWARE TRANSLATION

🔤 Testing: 'ACCOUNT'
----------------------------------------
🔄 Context-Enhanced Translation: 'ACCOUNT' -> 'da'
  📚 Context: 'Bank account' -> 'Bankkonto' (sim: 0.488)
  📚 Context: 'Don't have an account?' -> 'Har du ikke en konto?' (sim: 0.712)
  🤖 Using NLLB with 2 context examples
  📝 Enhanced input: Bank account means Bankkonto. Don't have an account? means Har du ikke en konto?. Translate ACCOUNT ...
  ✅ Context-enhanced result: 'الحساب المصرفي يعني حساب مصرفي. ليس لديك حساب؟ يعني Har du ikke en konto?. ترجمة حساب إلى da'
✅ Result: 'الحساب المصرفي يعني حساب مصرفي. ليس لديك حساب؟ يعني Har du ikke en konto?. ترجمة حساب إلى da'

🔤 Testing: 'WITHDRAWAL'
----------------------------------------
🔄 Context-Enhanced Translation: 'WITHDRAWAL' -> 'da'
  📚 Context: 'Withdrawal' -> 'Udtræk' (sim: 0.000)
  📚 Context: 'Withdrawals' -> 'Udbetalinger' (sim: 0.131)
  🎯 Exact match found: 'Udtræk'
✅ Result: 'Udtræk'

🔤 Testing: 'Craft Creditor'
--------

In [20]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI(
    title="NLLB Translation API with RAG",
    description="Context-aware translation using NLLB model and FAISS vector search",
    version="1.0.0"
)

# Add CORS middleware to allow frontend access
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], 
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class TranslationRequest(BaseModel):
    text: str
    target_language: str = "da"  # Default to Danish

class TranslationResponse(BaseModel):
    translatedText: str
    language: str
    context_used: bool = False

@app.post("/translate", response_model=TranslationResponse)
def translate(request: TranslationRequest):
    try:
        # Extract text from request
        text_to_translate = request.text
        target_language = request.target_language

        if not text_to_translate:
            raise HTTPException(status_code=400, detail="No text found to translate")
        
        # Use translation service to translate text - Use the final fixed function
        translated_text = rag_translate_final(
            input_text=text_to_translate,
            target_lang_short=target_language  
        )
        
        return TranslationResponse(
            translatedText=translated_text,
            language=target_language,
            context_used=True  # Assume context was attempted
        )
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Translation failed: {str(e)}")

@app.get("/health")
def health_check():
    return {"message": "API is running"}

@app.get("/languages")
def get_supported_languages():
    """Get list of supported languages"""
    return {
        "supported_languages": {
            "da": "Danish",
            "es": "Spanish", 
            "fi": "Finnish",
            "no": "Norwegian",
            "sv": "Swedish",
            "de": "German",
            "fr": "French",
            "ru": "Russian",
            "et": "Estonian",
            "lv": "Latvian"
        }
    }

In [21]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

# Create NEW FastAPI app with fixed translation
app_fixed = FastAPI(
    title="AI Translator with RAG - Fixed",
    description="Clean context-aware translation API using NLLB-200 model",
    version="2.1.0"
)

# Add CORS middleware
app_fixed.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app_fixed.get("/")
def read_root():
    return {"message": "AI Translator with RAG API - Fixed", "status": "running", "version": "2.1.0"}

@app_fixed.get("/health")
def health_check():
    return {"status": "healthy", "model": "NLLB-200", "features": ["RAG", "clean-translation"]}

@app_fixed.post("/translate", response_model=TranslationResponse)
def translate_fixed(request: TranslationRequest):
    """
    Translate text using the best translation function (rag_translate_final)
    """
    try:
        # Use the CLEAN translation function that actually works
        translated_text = rag_translate_final(request.text, request.target_language)
        
        return TranslationResponse(
            original_text=request.text,
            translated_text=translated_text,
            target_language=request.target_language,
            context_used=True  # rag_translate_final uses context when available
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Translation error: {str(e)}")

print("🔧 Updated FastAPI app to use rag_translate_final!")
print("✅ This should produce clean, accurate Danish translations")
print("")

# # Start the fixed server on port 8002
# def start_fixed_server():
#     """Start the FIXED FastAPI server on port 8002"""
#     uvicorn.run(app_fixed, host="0.0.0.0", port=8002, log_level="info")

# print("🚀 Starting FIXED FastAPI server on port 8002...")

# # Start server in background thread
# server_thread_fixed = threading.Thread(target=start_fixed_server, daemon=True)
# server_thread_fixed.start()

# # Wait for server to start
# time.sleep(3)

# # Test the fixed server
# try:
#     response = requests.get("http://localhost:8002/health")
#     if response.status_code == 200:
#         print("✅ FIXED FastAPI server started successfully on port 8002!")
#         print("🌐 Server running at: http://localhost:8002")
#         print("📚 API documentation: http://localhost:8002/docs")
#         print("")
#         print("🧪 Testing FIXED translation endpoint...")
        
#         # Test the translation endpoint with all previous test cases
#         test_cases = ["Good morning", "How are you?", "Thank you", "Goodbye"]
        
#         for test_text in test_cases:
#             test_data = {
#                 "text": test_text,
#                 "target_language": "da"
#             }
            
#             response = requests.post("http://localhost:8002/translate", json=test_data)
#             if response.status_code == 200:
#                 result = response.json()
#                 print(f"✅ '{test_text}' → '{result['translated_text']}'")
#             else:
#                 print(f"❌ '{test_text}' → FAILED: {response.status_code}")
        
#     else:
#         print(f"⚠️ Server health check failed: {response.status_code}")
# except Exception as e:
#     print(f"❌ Server connection failed: {e}")

# print("")
# print("🎉 FINAL API READY!")
# print("🌐 Use port 8002 for clean, accurate translations")
# print("📱 Perfect for public exposure via VS Code port forwarding!")

🔧 Updated FastAPI app to use rag_translate_final!
✅ This should produce clean, accurate Danish translations



In [22]:
# # Start the FastAPI server
# import uvicorn
# import threading
# import time

# def start_api_server():
#     """Start the FastAPI server in a separate thread"""
#     uvicorn.run(app, port=8000, log_level="info")

# # Start server in background thread
# print("🚀 Starting FastAPI server...")
# server_thread = threading.Thread(target=start_api_server, daemon=True)
# server_thread.start()

# # Give server time to start
# time.sleep(3)
# print("✅ FastAPI server started!")
# print("🌐 API is available at: http://localhost:8000")
# print("📚 Interactive docs at: http://localhost:8000/docs")
# print("📖 ReDoc docs at: http://localhost:8000/redoc")

INFO:     Started server process [29600]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


🚀 Starting FastAPI server...
✅ FastAPI server started!
🌐 API is available at: http://localhost:8000
📚 Interactive docs at: http://localhost:8000/docs
📖 ReDoc docs at: http://localhost:8000/redoc
✅ FastAPI server started!
🌐 API is available at: http://localhost:8000
📚 Interactive docs at: http://localhost:8000/docs
📖 ReDoc docs at: http://localhost:8000/redoc


INFO:     127.0.0.1:61728 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61833 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61833 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61871 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61871 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61882 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61882 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61894 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61894 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61904 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61904 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61910 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61910 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61919 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61919 - "POST /translate HTTP/1.1" 200 OK
INFO:     127.0.0.1:61933 - "POST /translate HTTP/1.1" 200 OK
INFO:   

In [23]:
# # 🚀 START FastAPI SERVER (with port conflict handling)
# import uvicorn
# import threading
# import time
# import requests
# import psutil
# import socket

# def find_available_port(start_port=8000, max_port=8010):
#     """Find an available port starting from start_port"""
#     for port in range(start_port, max_port):
#         try:
#             with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#                 s.bind(('localhost', port))
#                 return port
#         except OSError:
#             continue
#     return None

# def kill_process_on_port(port):
#     """Kill any process using the specified port"""
#     try:
#         for proc in psutil.process_iter(['pid', 'name', 'connections']):
#             try:
#                 for conn in proc.info['connections'] or []:
#                     if conn.laddr.port == port:
#                         print(f"   🔄 Killing process {proc.info['pid']} ({proc.info['name']}) using port {port}")
#                         proc.kill()
#                         time.sleep(1)
#                         return True
#             except (psutil.AccessDenied, psutil.NoSuchProcess):
#                 continue
#     except Exception as e:
#         print(f"   ⚠️ Could not kill process on port {port}: {e}")
#     return False

# def start_fastapi_server(port=8000, force_restart=False):
#     """Start FastAPI server with proper error handling"""
#     print("🚀 STARTING FASTAPI SERVER")
#     print("=" * 40)
    
#     # Check if server is already running
#     try:
#         response = requests.get(f"http://127.0.0.1:{port}/health", timeout=3)
#         if response.status_code == 200:
#             print(f"✅ Server already running on port {port}!")
#             print(f"🌐 API URL: http://127.0.0.1:{port}")
#             print(f"📖 Docs URL: http://127.0.0.1:{port}/docs")
#             return port
#     except:
#         pass
    
#     # Check if port is in use
#     try:
#         with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#             s.bind(('localhost', port))
#     except OSError:
#         print(f"⚠️ Port {port} is in use!")
        
#         if force_restart:
#             print("🔄 Attempting to free port...")
#             kill_process_on_port(port)
#             time.sleep(2)
#         else:
#             # Find alternative port
#             available_port = find_available_port(port + 1)
#             if available_port:
#                 print(f"📍 Using alternative port: {available_port}")
#                 port = available_port
#             else:
#                 print("❌ No available ports found!")
#                 return None
    
#     # Check if FastAPI app is defined
#     if 'app' not in globals():
#         print("❌ FastAPI app not defined!")
#         print("💡 Please run the FastAPI setup cells first")
#         return None
    
#     print(f"🎯 Starting server on port {port}...")
    
#     def run_server():
#         try:
#             uvicorn.run(
#                 app, 
#                 host="127.0.0.1", 
#                 port=port, 
#                 log_level="info",
#                 reload=False
#             )
#         except Exception as e:
#             print(f"❌ Server error: {e}")
    
#     # Start server in background thread
#     global server_thread
#     server_thread = threading.Thread(target=run_server, daemon=True)
#     server_thread.start()
    
#     # Wait for server to start
#     print("⏳ Waiting for server to start...")
#     for i in range(10):  # Wait up to 10 seconds
#         time.sleep(1)
#         try:
#             response = requests.get(f"http://127.0.0.1:{port}/health", timeout=2)
#             if response.status_code == 200:
#                 print("✅ Server started successfully!")
#                 print(f"🌐 API URL: http://127.0.0.1:{port}")
#                 print(f"📖 Docs URL: http://127.0.0.1:{port}/docs")
#                 print(f"🏥 Health: {response.json()}")
                
#                 # Test translation
#                 print("\n🧪 Quick translation test...")
#                 test_response = requests.post(
#                     f"http://127.0.0.1:{port}/translate",
#                     json={"text": "Hello", "target_language": "da"},
#                     timeout=10
#                 )
#                 if test_response.status_code == 200:
#                     result = test_response.json()
#                     print(f"✅ Translation test: '{result.get('translatedText', 'N/A')}'")
#                 else:
#                     print(f"⚠️ Translation test failed: {test_response.status_code}")
                
#                 return port
#         except:
#             print(f"   ⏳ Attempt {i+1}/10...")
    
#     print("❌ Server failed to start within 10 seconds")
#     return None

# # Start the server
# server_port = start_fastapi_server(port=8000, force_restart=True)

# 📚 Technology Stack Summary

## 🚀 Translation System with RAG Architecture

### Core Components

#### 🤖 **Machine Learning Models**
- **NLLB-200 (1.3B Distilled)**: Facebook's multilingual neural machine translation model
  - Supports 200+ languages
  - Optimized distilled version for efficiency
  - Local caching for improved performance

#### 🔍 **Retrieval-Augmented Generation (RAG)**
- **FAISS (Facebook AI Similarity Search)**: Vector similarity search engine
  - L2 distance metric for semantic similarity
  - Real-time context retrieval from knowledge base
- **Sentence Transformers**: Text embedding generation
  - Model: `all-MiniLM-L6-v2`
  - 384-dimensional embeddings for semantic search

#### 📊 **Knowledge Base**
- **Custom Translation Dictionary**: JSON-based multilingual glossary
- **Pandas DataFrame**: Structured data manipulation and filtering
- **Semantic Indexing**: Context-aware translation examples

### 🌐 **API Framework**
- **FastAPI**: Modern, fast web framework for building APIs
  - Automatic OpenAPI documentation
  - Type hints and validation with Pydantic
  - Async support for better performance
- **Uvicorn**: ASGI server for running FastAPI applications
- **RESTful Endpoints**: Standard HTTP methods for translation services

### 📦 **Python Ecosystem**
- **PyTorch**: Deep learning framework for model inference
- **Transformers (Hugging Face)**: State-of-the-art NLP model library
- **NumPy**: Numerical computing for vector operations
- **Requests**: HTTP client for API testing
- **JSON**: Data serialization and knowledge base storage

### 🔧 **Development Environment**
- **Jupyter Notebook**: Interactive development and testing
- **Virtual Environment**: Isolated Python package management
- **Windows PowerShell**: Command-line interface and automation

### 🏗️ **Architecture Pattern**
- **RAG (Retrieval-Augmented Generation)**: Combines semantic search with generative AI
- **Microservices**: API-first architecture for scalability
- **Local-First**: Models and data cached locally for privacy and speed
- **Context-Aware Translation**: Uses domain-specific examples for accuracy

### 🔒 **Key Features**
- **Multi-language Support**: 11 target languages (Danish, Spanish, Finnish, etc.)
- **Context-Aware**: Uses similar translation examples for better accuracy
- **Exact Match Detection**: Returns known translations immediately
- **Fallback Mechanisms**: Multiple strategies for handling edge cases
- **Real-time API**: Fast response times with local model inference
- **Interactive Documentation**: Auto-generated API docs with Swagger UI

### 📈 **Performance Optimizations**
- **Model Caching**: Local storage of downloaded models
- **Embedding Reuse**: Pre-computed vectors for knowledge base
- **Batch Processing**: Efficient handling of multiple translations
- **Memory Management**: Optimized model loading and inference

### 🛠️ **Deployment**
- **Local Development**: Notebook-based development and testing
- **API Server**: Production-ready FastAPI application
- **Port Configuration**: Configurable host and port settings
- **Health Monitoring**: Built-in health check endpoints

# 🌍 Public API Exposure with ngrok

## Setup Instructions for Public Access

### 1. Install ngrok
```bash
# Download ngrok from https://ngrok.com/download
# Or install via package manager:
# Windows (Chocolatey): choco install ngrok
# Or download the executable and add to PATH
```

### 2. Sign up and Get Auth Token
- Visit https://ngrok.com/signup
- Get your auth token from the dashboard
- Configure it: `ngrok config add-authtoken YOUR_TOKEN`

### 3. Expose Your API
- Run your local API server first
- Then use ngrok to create a public tunnel
- Share the ngrok URL for public access

### ⚠️ Security Considerations
- The API will be publicly accessible
- Consider adding authentication/API keys
- Monitor usage and costs
- ngrok free tier has limitations

# 🌐 Alternative Hosting Solutions for Your Translation API

Since you're experiencing issues with ngrok, here are excellent alternatives to temporarily host your FastAPI translation API publicly:

## 🎯 **Best Alternatives (Ranked by Ease)**

### 1. **Cloudflare Tunnel (cloudflared)** ⭐⭐⭐⭐⭐
- **Why it's great**: More reliable than ngrok, better performance, no HTML landing pages
- **Free tier**: Generous limits, no time restrictions
- **Setup**: Very simple, single binary download
- **Best for**: Production-like testing, reliable public access

### 2. **Localhost.run** ⭐⭐⭐⭐
- **Why it's great**: Zero setup, works immediately via SSH
- **Free tier**: Completely free, no registration needed
- **Setup**: One SSH command
- **Best for**: Quick testing, no installation required

### 3. **LocalTunnel** ⭐⭐⭐⭐
- **Why it's great**: NPM package, very simple
- **Free tier**: Free with optional custom subdomains
- **Setup**: Simple npm install
- **Best for**: If you have Node.js installed

### 4. **Serveo.net** ⭐⭐⭐
- **Why it's great**: SSH-based, no installation
- **Free tier**: Completely free
- **Setup**: One SSH command
- **Best for**: Quick testing, SSH available

### 5. **Railway** ⭐⭐⭐⭐⭐
- **Why it's great**: Professional deployment platform
- **Free tier**: $5 credit monthly
- **Setup**: Git-based deployment
- **Best for**: Longer-term hosting, professional demos

## 🔧 **Implementation Guide**

Choose the option that works best for your situation!

# 🚀 VS Code Built-in Port Forwarding (EASIEST!)

## ✨ **Visual Studio Code has built-in tunneling!** 

This is **the easiest and most reliable method** - no external tools needed!

### 🎯 **Method 1: VS Code Port Forwarding (Recommended)**

1. **Open the Terminal panel** in VS Code (View → Terminal)
2. **Look for the "PORTS" tab** next to Terminal
3. **Click "PORTS"** tab
4. **Click "+ Forward a Port"** 
5. **Enter: 8000** (your FastAPI port)
6. **Choose "Public"** when asked about visibility
7. **Copy the public URL** that appears!

### 🌟 **Benefits:**
- ✅ **No installation** required
- ✅ **No passwords** or warning screens  
- ✅ **Professional URLs** (*.vscode.dev)
- ✅ **Reliable connection**
- ✅ **Easy to manage** from VS Code interface

### 📱 **Alternative: VS Code Tunnels**
If port forwarding isn't available, use VS Code Tunnels:
1. **Command Palette** (Ctrl+Shift+P)
2. **Type: "tunnel"**  
3. **Select "Turn on Remote Tunnel Access"**
4. **Follow the GitHub authentication**
5. **Get a permanent tunnel URL**

---

## 🔄 **Step-by-Step Visual Guide:**

In [24]:
# # 🎯 VS CODE PORT FORWARDING - Step by Step Guide

# def check_fastapi_running():
#     """Check if FastAPI is running before setting up port forwarding"""
#     print("🏥 CHECKING FASTAPI STATUS")
#     print("=" * 30)
    
#     try:
#         import requests
#         response = requests.get("http://localhost:8000/health", timeout=5)
#         if response.status_code == 200:
#             print("✅ FastAPI is running on port 8000")
#             print(f"   Health response: {response.json()}")
#             return True
#         else:
#             print(f"⚠️ FastAPI responding with status: {response.status_code}")
#             return False
#     except Exception as e:
#         print(f"❌ FastAPI not accessible on localhost:8000")
#         print(f"   Error: {e}")
#         print("💡 Make sure you've started the FastAPI server first!")
#         return False

# def provide_vscode_instructions():
#     """Provide step-by-step VS Code port forwarding instructions"""
#     print("\n🚀 VS CODE PORT FORWARDING INSTRUCTIONS")
#     print("=" * 45)
    
#     print("📋 STEP-BY-STEP GUIDE:")
#     print("")
#     print("1. 👀 LOOK AT THE BOTTOM PANEL of VS Code")
#     print("   - You should see tabs: TERMINAL, OUTPUT, DEBUG CONSOLE, etc.")
#     print("")
#     print("2. 🔍 FIND THE 'PORTS' TAB")
#     print("   - Click on the 'PORTS' tab (next to TERMINAL)")
#     print("   - If you don't see it, go to View → Command Palette")
#     print("   - Type: 'View: Toggle Panel' and press Enter")
#     print("")
#     print("3. ➕ ADD PORT FORWARDING")
#     print("   - In the PORTS panel, click '+ Forward a Port'")
#     print("   - OR click the 'Forward a Port' button")
#     print("")
#     print("4. 📝 ENTER PORT NUMBER")
#     print("   - Type: 8000")
#     print("   - Press Enter")
#     print("")
#     print("5. 🌐 SET VISIBILITY TO PUBLIC")
#     print("   - Right-click on the port entry")
#     print("   - Select 'Port Visibility → Public'")
#     print("   - OR look for a globe icon and click it")
#     print("")
#     print("6. 📋 COPY THE PUBLIC URL")
#     print("   - You'll see a URL like: https://xyz-8000.app.github.dev")
#     print("   - Right-click → 'Copy Local Address' or 'Copy Public URL'")
#     print("")
#     print("7. 🧪 TEST YOUR API")
#     print("   - Open the URL in your browser")
#     print("   - Add '/docs' to see the API documentation")
#     print("   - Example: https://xyz-8000.app.github.dev/docs")

# def provide_alternative_method():
#     """Provide alternative VS Code tunnel method"""
#     print("\n🔄 ALTERNATIVE: VS CODE REMOTE TUNNELS")
#     print("=" * 35)
#     print("If port forwarding doesn't work:")
#     print("")
#     print("1. 🎮 OPEN COMMAND PALETTE")
#     print("   - Press: Ctrl+Shift+P (Windows) or Cmd+Shift+P (Mac)")
#     print("")
#     print("2. 🔍 SEARCH FOR TUNNEL")
#     print("   - Type: 'Remote-Tunnels: Turn on Remote Tunnel Access'")
#     print("   - Select it from the dropdown")
#     print("")
#     print("3. 🔐 AUTHENTICATE WITH GITHUB")
#     print("   - Follow the prompts to sign in with GitHub")
#     print("   - This creates a permanent tunnel to your machine")
#     print("")
#     print("4. 🌐 GET YOUR TUNNEL URL")
#     print("   - You'll get a URL like: https://xyz.tunnels.api.visualstudio.com")
#     print("   - Your API will be available at: [tunnel-url]:8000")

# def provide_troubleshooting():
#     """Provide troubleshooting tips"""
#     print("\n🔧 TROUBLESHOOTING")
#     print("=" * 15)
#     print("If you don't see the PORTS tab:")
#     print("• View → Command Palette → 'View: Toggle Panel'")
#     print("• Or press Ctrl+` (backtick) to toggle terminal panel")
#     print("")
#     print("If port forwarding option is missing:")
#     print("• Update VS Code to the latest version")
#     print("• Make sure you're using VS Code (not VS Code Insiders)")
#     print("")
#     print("If the URL doesn't work:")
#     print("• Make sure FastAPI is running (check step 1)")
#     print("• Try setting port visibility to 'Public'")
#     print("• Wait 30 seconds and try again")

# # Run the checks and provide instructions
# is_running = check_fastapi_running()

# if is_running:
#     print("\n🎉 READY FOR PORT FORWARDING!")
#     provide_vscode_instructions()
# else:
#     print("\n⚠️ START FASTAPI FIRST!")
#     print("Run the FastAPI server cell before setting up port forwarding")

# provide_alternative_method()
# provide_troubleshooting()

In [25]:
# # 🚀 START FASTAPI + VS CODE PORT FORWARDING DEMO

# import uvicorn
# import threading
# import time
# import requests

# def start_fastapi_for_vscode():
#     """Start FastAPI server specifically for VS Code port forwarding"""
#     print("🚀 STARTING FASTAPI FOR VS CODE PORT FORWARDING")
#     print("=" * 50)
    
#     # Check if already running
#     try:
#         response = requests.get("http://localhost:8000/health", timeout=2)
#         if response.status_code == 200:
#             print("✅ FastAPI already running!")
#             print("🎯 Ready for VS Code port forwarding!")
#             return True
#     except:
#         pass
    
#     print("🔄 Starting FastAPI server...")
    
#     def run_server():
#         uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info")
    
#     # Start server in background thread
#     server_thread = threading.Thread(target=run_server, daemon=True)
#     server_thread.start()
    
#     # Wait for server to start
#     print("⏳ Waiting for server to start...")
#     max_attempts = 10
#     for attempt in range(max_attempts):
#         try:
#             response = requests.get("http://localhost:8000/health", timeout=2)
#             if response.status_code == 200:
#                 print("✅ FastAPI server started successfully!")
#                 print(f"   Health check: {response.json()}")
#                 return True
#         except:
#             if attempt < max_attempts - 1:
#                 time.sleep(2)
            
#     print("❌ Server failed to start")
#     return False

# def demonstrate_vscode_steps():
#     """Show the exact VS Code steps with visual cues"""
#     print("\n🎯 NOW FOLLOW THESE VS CODE STEPS:")
#     print("=" * 40)
    
#     print("👀 STEP 1: LOOK AT THE BOTTOM OF VS CODE")
#     print("   ▼ You should see these tabs:")
#     print("   [TERMINAL] [OUTPUT] [PORTS] [DEBUG CONSOLE]")
#     print("")
    
#     print("🖱️ STEP 2: CLICK THE 'PORTS' TAB")
#     print("   ▼ In the PORTS panel, you'll see:")
#     print("   [+ Forward a Port] [⚙️ Configure]")
#     print("")
    
#     print("➕ STEP 3: CLICK '+ Forward a Port'")
#     print("   ▼ A popup will appear asking for port number")
#     print("   [Enter port number: ____]")
#     print("   Type: 8000")
#     print("")
    
#     print("🌐 STEP 4: SET TO PUBLIC")
#     print("   ▼ Right-click on the new port entry")
#     print("   [Port Visibility → 🌍 Public]")
#     print("")
    
#     print("📋 STEP 5: COPY THE URL")
#     print("   ▼ You'll see something like:")
#     print("   [🌍 https://xyz-8000.app.github.dev] [📋 Copy]")
#     print("")
    
#     print("🧪 STEP 6: TEST YOUR API")
#     print("   ▼ Open the URL and add '/docs':")
#     print("   https://xyz-8000.app.github.dev/docs")

# def provide_quick_summary():
#     """Quick summary of the process"""
#     print("\n⚡ QUICK SUMMARY:")
#     print("=" * 15)
#     print("1. ✅ FastAPI is running on localhost:8000")
#     print("2. 👀 Look for PORTS tab at bottom of VS Code")
#     print("3. ➕ Click '+ Forward a Port' → Enter 8000")
#     print("4. 🌐 Set visibility to Public")
#     print("5. 📋 Copy the public URL")
#     print("6. 🧪 Test: [your-url]/docs")
#     print("")
#     print("🎉 This is the EASIEST method - no external tools!")

# # Start FastAPI
# success = start_fastapi_for_vscode()

# if success:
#     demonstrate_vscode_steps()
#     provide_quick_summary()
# else:
#     print("\n❌ Please check the FastAPI startup cell and try again")

🔄 Translating: 'code' to 'da'
  🤖 Using NLLB with context-enhanced translation
  📚 Context prompt: Examples: Enter code -> Indtast kode. Translate: code
  ✅ NLLB context-enhanced translation: Indtast kode
  ✅ NLLB context-enhanced translation: Indtast kode
